In [1]:
import numpy as np
import matplotlib.pyplot as plt
import qutip as qt
import scipy as sp

In [2]:
sx = qt.operators.sigmax()
sy = qt.operators.sigmay()

In [3]:
sx-1j*sy

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
Qobj data =
[[0. 0.]
 [2. 0.]]

In [5]:
def hamiltonian_ising(N,omega,delta,V):
  sx = qt.operators.sigmax()
  sy = qt.operators.sigmay()
  sz = qt.operators.sigmaz()
  I = qt.operators.identity(2)

  ising_ham = qt.tensor([I]*N)
  for i in range(N):
    hz = qt.tensor([sz if j==i else I for j in range(N)])
    hx = qt.tensor([sx if j==i else I for j in range(N)])
    for j in range(i + 1, N):
        # Interaction term between qubit i and qubit j
        hzz = qt.tensor([sz if k == i or k == j else I for k in range(N)]) # assume obc
    ising_ham += -1*delta*hz+omega*hx+(V/N)*hzz
    return(ising_ham)

In [6]:
omega = 1
N = 6
delta = 1
V = 3
hamiltonian=hamiltonian_ising(N,omega,delta,V)

In [7]:
sm = qt.operators.sigmam()
I = qt.operators.identity(2)
kappa = 1
c_ops = [qt.tensor([sm if i==j  else I for j in range(N)]) for i in range(N)]
c_ops = [np.sqrt(kappa) * c_op for c_op in c_ops]
#L = qt.liouvillian(hamiltonian, c_ops).full()
L_dag = qt.liouvillian(hamiltonian, c_ops).dag().full()

In [8]:
sp.sparse.linalg.eigsh(L_dag,4,which='SM')

(array([ 1.15669953e-15, -5.00000000e-01, -5.00000000e-01, -5.00000000e-01]),
 array([[ 8.98895871e-02-8.68611658e-02j,  6.12533686e-16-1.27713709e-15j,
          9.36951808e-16+2.32998690e-17j,  1.16802362e-16-6.40576699e-17j],
        [ 5.84669708e-16+3.32404431e-16j,  5.23326781e-16+7.28618273e-17j,
          5.01916602e-16-2.62170347e-16j,  6.89422884e-17+8.65298617e-17j],
        [-2.42890327e-16-1.09415076e-16j,  5.54363359e-02+4.69528317e-04j,
          2.11867953e-02+1.00707408e-02j,  4.47568894e-02-3.32939094e-02j],
        ...,
        [ 3.06807440e-16-2.13305932e-16j, -3.23727916e-02+6.35739007e-02j,
          6.75641749e-02-8.05840913e-02j,  5.53474195e-02+5.89814511e-03j],
        [ 3.21580039e-16-6.90270687e-17j,  2.18612493e-16+1.32015460e-16j,
          1.17897297e-17+3.20410507e-17j,  1.56109837e-16-3.33093389e-17j],
        [ 8.98895871e-02-8.68611658e-02j,  1.84680185e-16-2.85700535e-16j,
          2.91238876e-16-1.03736839e-16j,  4.50515897e-17-4.31845673e-17j]]))

In [9]:
val,vec = sp.sparse.linalg.eigsh(L_dag,2,which='SM')
l2 = vec[:,1]

In [10]:
l2 = np.matrix(l2.reshape(2**N,2**N))
l2d = l2.H

l2*l2d

matrix([[ 1.61221601e-02+0.00000000e+00j,
          2.01162926e-18+1.41526487e-19j,
         -2.06635439e-18+5.61330212e-17j, ...,
         -2.08300252e-19-6.01724807e-21j,
          9.51085526e-21-7.42781118e-19j,
          3.10446849e-19+1.28910234e-19j],
        [ 2.01162926e-18-1.41526487e-19j,
          1.61221601e-02+0.00000000e+00j,
         -2.71458906e-17-1.81567731e-17j, ...,
         -1.45167434e-18-5.33802090e-18j,
          2.75524899e-19-1.04269158e-19j,
          1.36115401e-18-1.09681791e-18j],
        [-2.06635439e-18-5.61330212e-17j,
         -2.71458906e-17+1.81567731e-17j,
          2.03372459e-02+0.00000000e+00j, ...,
         -7.51048530e-20-8.03934840e-20j,
          5.30553182e-19-3.08864482e-18j,
          1.67540556e-19-3.64984282e-20j],
        ...,
        [-2.08300252e-19+6.01724807e-21j,
         -1.45167434e-18+5.33802090e-18j,
         -7.51048530e-20+8.03934840e-20j, ...,
          1.09127541e-02+0.00000000e+00j,
         -4.75222888e-17+2.01905898e-17j

In [11]:
np.linalg.eig(l2.reshape(2**N,2**N))

EigResult(eigenvalues=array([ 0.13465347-0.15836682j, -0.13465347+0.15836682j,
       -0.13465347+0.15836682j, -0.13465347+0.15836682j,
        0.13465347-0.15836682j,  0.13465347-0.15836682j,
        0.13465347-0.15836682j, -0.13465347+0.15836682j,
        0.02035905-0.13686046j,  0.02035905-0.13686046j,
        0.02035905-0.13686046j,  0.02035905-0.13686046j,
       -0.02035905+0.13686046j, -0.02035905+0.13686046j,
       -0.02035905+0.13686046j, -0.02035905+0.13686046j,
        0.09949589+0.05993148j,  0.09949589+0.05993148j,
        0.09949589+0.05993148j,  0.09949589+0.05993148j,
       -0.09949589-0.05993148j, -0.09949589-0.05993148j,
       -0.09949589-0.05993148j, -0.09949589-0.05993148j,
        0.11869186-0.05298587j,  0.11869186-0.05298587j,
        0.11869186-0.05298587j,  0.11869186-0.05298587j,
        0.1154575 -0.04544946j,  0.1154575 -0.04544946j,
        0.1154575 -0.04544946j,  0.1154575 -0.04544946j,
       -0.11869186+0.05298587j, -0.11869186+0.05298587j,
       -0

In [ ]:
psi0 = qt.tensor(qt.basis(2,0),qt.basis(2,1))
final_state = qt.steadystate(hamiltonian,c_ops)
#final_state.full() # density matrix corresponding to zero eigen value

tlist = omega*np.linspace(0, 10, 100)
medata = qt.mesolve(hamiltonian, psi0, tlist, c_ops, [])

In [ ]:
hilbert_distance = []

for state in medata.states:
  dis = qt.hilbert_dist(state,final_state)
  hilbert_distance.append(dis)

plt.plot(tlist,hilbert_distance)
plt.xlabel('time')
plt.ylabel('hilbert distance')
plt.show()